In [ ]:
import numpy as np
import pandas as pd

import os
import random

import warnings

warnings.filterwarnings(action='ignore')

path = '/opt/ml/input/data/'
dat = pd.read_csv(path + '/FE_total2.csv')


In [ ]:
dat.drop(['day','year','KnowledgeTag', 'last_answerCode7','last_answerCode8','last_answerCode9','last_answerCode10'], axis = 1, inplace = True)

In [ ]:
_train = dat[dat['answerCode'] >= 0]
_test = dat[dat['answerCode'] < 0]

In [ ]:
# # valid 제작하는 함수 새로 개편.
# # 예전 valid 제작 함수는 1분 걸렸는데 0.6초만에 끗~
# _train['train_valid'] = 0
# _train.loc[_train.drop_duplicates(subset='userID', keep = 'last').index, 'train_valid'] = -1
# _valid = _train[_train['train_valid'] == -1]
# _train = _train[_train['train_valid'] == 0]

In [ ]:
# 모델에 적용하기 전 기본적인 데이터 전처리 부분
## 라벨링, 필요없는 칼럼 제거
_train_value = _train['answerCode']
_train.drop(['Timestamp', 'testId'], axis = 1, inplace = True) # ,'category_st_qcut_5'

# _valid_value = _valid['answerCode']
# _valid.drop(['Timestamp', 'testId', 'train_valid', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'

_test.drop(['Timestamp', 'testId'], axis = 1, inplace = True) # ,'category_st_qcut_5'

In [ ]:
# CatBoost에 적용하기 위해선 문자열 데이터로 변환 필요.
# 만약 범주형이 아니고 연속형(정수, 소수) 성질을 살리고 싶은 변수는 여기 drop에 적어두면 됨.(solve_time 같이.)
cat_columns = list(_train.drop(['solve_time','month'],axis=1).columns)

for col in cat_columns:
    _train[col] = _train[col].astype('str')
    # _valid[col] = _valid[col].astype('str')
    _test[col] = _test[col].astype('str')

In [ ]:
def train_valid_split(df, train_size=0.90):
    # Total user number
    total_user = df.userID.nunique()

    # Total user indices
    total_user_index = df.userID.unique()

    # Divide the user number with ratio
    train_user = round(total_user * train_size)
    valid_user = total_user - train_user

    # Find the first user which located at validation set
    valid_user_id = total_user_index[train_user]

    # Find the first index of the validation set
    valid_index = df.query('userID == @valid_user_id').index[0]

    # Split the dataset to train and valid
    train = df.iloc[:valid_index]
    valid = df.iloc[valid_index:]

    return train, valid

In [ ]:
FEATHER = ['userID',
 'assessmentItemID',
 'b_category',
 'test_category',
 'problem_id',
 'category_st_qcut_5',
 'last_answerCode',
 'last_answerCode2',
 'last_answerCode3',
 'last_answerCode4',
 'last_answerCode5',
 'last_answerCode6',
 'hour']

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold

def objective(trial, train, FEATHER):

    param = {
    "task_type" : "GPU",
    "eval_metric" : 'AUC',
    # "devices" : '0',
    "random_state": 1998,
    'learning_rate': 0.01,
    # 'bagging_temperature': 0.05,
    'iterations': 4000,
    'max_depth': trial.suggest_int("max_depth", 8, 12),
    # 'random_strength': 0,
    # 'l2_leaf_reg': 2e-7,
    # 'min_child_samples': 5, 
    # 'max_bin': 100, 
    # 'od_type': 'Iter',
    "cat_features" : FEATHER,
    }
    
    skf = KFold(n_splits=3)
    roc_auc = []
    for _ in range(10):
        _train, _valid = train_valid_split(train)   
        _train_value = _train['answerCode']
        _train.drop('answerCode', axis = 1, inplace = True)
        _valid_value = _valid['answerCode']
        _valid.drop('answerCode', axis = 1, inplace = True)
        
        # print(_train.head())  
        # print(_train_value.head())  
        # print(_valid.head())  
        # print(_valid_value.head())  
        model = CatBoostClassifier(**param)
        model.fit(_train, _train_value, early_stopping_rounds=100, verbose=200, eval_set=(_valid, _valid_value))
        train_pred = model.predict_proba(_train)[:,1]     
        valid_pred = model.predict_proba(_valid)[:,1]

        print(roc_auc_score(_valid_value, valid_pred))
        roc_auc.append(roc_auc_score(_valid_value, valid_pred))

        print('train score')
        print(roc_auc_score(_train_value, train_pred)) # auc
        print(accuracy_score(_train_value, np.where(train_pred >= 0.5, 1, 0))) # acc, 정확도

        print('valid score')
        print(roc_auc_score(_valid_value, valid_pred)) # auc
        print(accuracy_score(_valid_value, np.where(valid_pred >= 0.5, 1, 0))) # acc, 정확도   
    
    print(roc_auc)
    return sum(roc_auc) / 3

***optuna + kfold training***

In [ ]:
import optuna
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'maximize',
    sampler = sampler,
)
study.optimize(lambda trial : objective(trial, _train, FEATHER), n_trials=5)

***best param + kfold***

In [ ]:
from sklearn.model_selection import StratifiedKFold
_test['prediction'] = 0
skf = StratifiedKFold(n_splits=10)

for train_idx, value_idx in skf.split(_train, _train_value):   
    train = _train.iloc[train_idx, :]     
    train_value = _train_value.iloc[train_idx]  
    valid = _train.iloc[value_idx, :]   
    valid_value = _train_value.iloc[value_idx]  
    model = CatBoostClassifier(**study.best_params)
    model.fit(train[FEATURE], train_value, early_stopping_rounds=100, verbose=200, eval_set=(valid, valid_value))     
    test_pred = model.predict_proba(_test[FEATURE])[:,1]
    _test['prediction'] += test_pred    
    print(f'================================================================================\n\n')
    
_test['prediction'] /= 10
submission = _test['prediction'].reset_index(drop = True).reset_index()
submission.rename(columns = {'index':'id'}, inplace = True)
submission.to_csv(os.path.join('/opt/ml/input/code/level2_dkt_recsys-level2-recsys-04/output/submission.csv'), index = False)
print("SAVE COMPLETE")

In [ ]:
import matplotlib.pyplot as plt

feature_importance = cat_model.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align="center")
plt.yticks(range(len(sorted_idx)), np.array(_valid.columns)[sorted_idx])
plt.title("Feature Importance")

In [ ]:
# submission 제출하기 위한 코드

_test_pred = cat_model.predict_proba(_test)[:,1]
_test['prediction'] = _test_pred
submission = _test['prediction'].reset_index(drop = True).reset_index()
submission.rename(columns = {'index':'id'}, inplace = True)
submission.to_csv('../output/cat3.csv', index = False)